In [187]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
import glob
image_list = glob.glob('/content/gdrive/My Drive/VisDrone2019-DET-val/VisDrone2019-DET-val/images/*.jpg')

In [0]:
txt_list = glob.glob('/content/gdrive/My Drive/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations/*.txt')

In [191]:
import torch 
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
ssd_model.to('cuda')
ssd_model.eval()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [0]:
uris = image_list
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs, precision == 'fp16')

In [0]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

In [0]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.00) for results in results_per_input]
classes_to_labels = utils.get_coco_object_dictionary()

In [0]:
import pandas as pd
import numpy as np

In [0]:
def annots(txt):
  data = []
  data1 = []
  DATA = pd.DataFrame()
  for f in txt:
    data.append(np.genfromtxt(f, delimiter=',', dtype=np.float))
    data1.append(f[f.rindex('/')+1:])
  for i in range(len(data1)):
    df = pd.DataFrame(np.matrix(data[i]),columns=['bbox_left','bbox_top','bbox_width','bbox_height','score','object_category','truncation','occlusion'])
    df2 = pd.Series(np.full(len(np.matrix(data[i])),data1[i]))  
    df['name_file'] = df2
    DATA = DATA.append(df, ignore_index = True)
  return DATA

In [0]:
df_annots = annots(txt_list)

In [0]:
df_annots

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file
0,879.0,483.0,10.0,31.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt
1,854.0,479.0,8.0,18.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt
2,620.0,445.0,63.0,35.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt
3,579.0,406.0,61.0,30.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt
4,48.0,489.0,104.0,42.0,1.0,4.0,0.0,2.0,0000249_00001_d_0000001.txt
...,...,...,...,...,...,...,...,...,...
40164,709.0,634.0,117.0,51.0,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt
40165,1046.0,577.0,101.0,43.0,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt
40166,1232.0,580.0,103.0,45.0,1.0,4.0,0.0,1.0,0000316_01201_d_0000525.txt
40167,1305.0,519.0,54.0,35.0,1.0,8.0,1.0,1.0,0000316_01201_d_0000525.txt


In [0]:
def func_annots(txt, best_results):
  data = []
  data1 = []
  DATA = pd.DataFrame()
  for f in txt:
    data1.append(f[f.rindex('/')+1:])
  for i in range(len(data1)):
    for j in range(len(best_results[i][0])):
      left = best_results[i][0][j][0]
      bot = best_results[i][0][j][1]
      right = best_results[i][0][j][2]
      top = best_results[i][0][j][3]
      best_results[i][0][j][0] = 300 * left
      best_results[i][0][j][1] = 300 * bot
      best_results[i][0][j][2] = 300 * (right - left)
      best_results[i][0][j][3] = 300 * (top - bot)
    df1 = pd.DataFrame(best_results[i][0], columns=['bbox_left','bbox_top','bbox_width','bbox_height'])
    df2 = pd.Series(np.full(len(np.matrix(best_results[i][0])),data1[i]))
    df1['classes'] = pd.Series(best_results[i][1])
    df1['confidences'] = pd.Series(best_results[i][2])
    df1['name_file'] = df2 
    DATA = DATA.append(df1, ignore_index = True)
  return DATA

In [0]:
df_func_annots = func_annots(txt_list, best_results_per_input)

In [0]:
df_func_annots

,bbox_left,bbox_top,bbox_width,bbox_height,classes,confidences,name_file
0,42818.144531,42424.187500,-40824.445312,-36891.281250,1,0.105840,0000249_00001_d_0000001.txt
1,75974.328125,34902.578125,-74672.906250,-31739.785156,1,0.109001,0000249_00001_d_0000001.txt
2,58733.382812,39484.761719,-56213.648438,-34637.328125,1,0.111698,0000249_00001_d_0000001.txt
3,85474.453125,30829.177734,-84210.281250,-27768.492188,1,0.113531,0000249_00001_d_0000001.txt
4,15856.455078,22007.041016,-7068.621094,-18338.224609,14,0.116739,0000249_00001_d_0000001.txt
...,...,...,...,...,...,...,...
10767,64876.777344,52709.902344,-58076.558594,-45253.468750,68,0.560586,0000345_07057_d_0000345.txt
10768,42652.316406,65296.417969,-33341.375000,-54552.656250,3,0.682732,0000345_07057_d_0000345.txt
10769,48518.757812,67237.382812,-38138.519531,-54956.160156,3,0.848030,0000345_07057_d_0000345.txt
10770,31970.373047,62142.351562,-23055.015625,-52151.414062,68,0.908687,0000345_07057_d_0000345.txt


In [0]:
from PIL import Image 
import pandas as pd
import numpy as np

In [0]:
df_size = pd.DataFrame(columns=['width','height'])
for i in range(len(image_list)):
  im = Image.open(image_list[i])
  width, height = im.size
  df_size = df_size.append({'width': width, 'height': height, 'coeff': 300/height}, ignore_index=True)

In [0]:
data1 = []
for f in txt_list:
  data1.append(f[f.rindex('/')+1:])

In [0]:
df_size['name_file'] = data1

In [0]:
df_annots_new = pd.DataFrame()

In [0]:
df_annots_new = df_annots.copy()

In [0]:
df_annots_new = df_annots_new.merge(df_size,how='left',left_on = 'name_file',right_on = 'name_file')

In [0]:
df_annots_new.head()

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff
0,879.0,483.0,10.0,31.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
1,854.0,479.0,8.0,18.0,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
2,620.0,445.0,63.0,35.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
3,579.0,406.0,61.0,30.0,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556
4,48.0,489.0,104.0,42.0,1.0,4.0,0.0,2.0,0000249_00001_d_0000001.txt,960.0,540.0,0.555556


In [0]:
df_annots_new['bbox_left_x'] = df_annots_new.bbox_left * df_annots_new.coeff
df_annots_new['bbox_top'] = df_annots_new.bbox_top * df_annots_new.coeff
df_annots_new['bbox_width'] = df_annots_new.bbox_width * df_annots_new.coeff
df_annots_new['bbox_height'] = df_annots_new.bbox_height * df_annots_new.coeff
df_annots_new['width'] = df_annots_new.width * df_annots_new.coeff
df_annots_new['height'] = df_annots_new.height * df_annots_new.coeff

In [0]:
df_annots_new['bbox_center'] = df_annots_new.width/2
df_annots_new['bbox_left'] = df_annots_new.bbox_center - 150
df_annots_new['bbox_right'] = df_annots_new.bbox_center + 150
df_annots_new['bbox_right_x'] = df_annots_new.bbox_left_x + df_annots_new.bbox_width

In [0]:
df_annots_new_1 = df_annots_new.copy()

In [0]:
df_annots_new_1

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
0,116.666667,268.333333,5.555556,17.222222,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,488.333333,266.666667,416.666667,493.888889
1,116.666667,266.111111,4.444444,10.000000,1.0,1.0,0.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,474.444444,266.666667,416.666667,478.888889
2,116.666667,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,344.444444,266.666667,416.666667,379.444444
3,116.666667,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,321.666667,266.666667,416.666667,355.555556
4,116.666667,271.666667,57.777778,23.333333,1.0,4.0,0.0,2.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,26.666667,266.666667,416.666667,84.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40164,116.666667,264.166667,48.750000,21.250000,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,295.416667,266.666667,416.666667,344.166667
40165,116.666667,240.416667,42.083333,17.916667,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,435.833333,266.666667,416.666667,477.916667
40166,116.666667,241.666667,42.916667,18.750000,1.0,4.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,513.333333,266.666667,416.666667,556.250000
40167,116.666667,216.250000,22.500000,14.583333,1.0,8.0,1.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,543.750000,266.666667,416.666667,566.250000


In [0]:
df_annots_new_2 = pd.DataFrame()

In [0]:
df_annots_new_2 = df_annots_new_1.loc[(df_annots_new_1.bbox_left_x >= df_annots_new_1.bbox_left) & (df_annots_new_1.bbox_left_x < df_annots_new_1.bbox_right) & (df_annots_new_1.bbox_right_x > df_annots_new_1.bbox_left) & (df_annots_new_1.bbox_right_x <= df_annots_new_1.bbox_right)]

In [0]:
df_annots_new_2 = pd.DataFrame()
for i in range(len(df_annots_new_1)):
  if ((df_annots_new_1.iloc[i,12]>=df_annots_new_1.iloc[i,0])&(df_annots_new_1.iloc[i,12]<df_annots_new_1.iloc[i,14])&(df_annots_new_1.iloc[i,15]>df_annots_new_1.iloc[i,0])&(df_annots_new_1.iloc[i,15]<=df_annots_new_1.iloc[i,14])):
    df_annots_new_2 = df_annots_new_2.append(df_annots_new_1.iloc[i])

In [0]:
df_annots_new_3 = df_annots_new_2.copy()

In [0]:
df_annots_new_3

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
2,116.666667,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,344.444444,266.666667,416.666667,379.444444
3,116.666667,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,321.666667,266.666667,416.666667,355.555556
6,116.666667,281.111111,7.222222,11.111111,1.0,2.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,168.333333,266.666667,416.666667,175.555556
7,116.666667,282.777778,9.444444,16.666667,1.0,10.0,1.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,167.222222,266.666667,416.666667,176.666667
8,116.666667,276.666667,45.555556,23.333333,1.0,4.0,1.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,287.777778,266.666667,416.666667,333.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40160,116.666667,223.333333,5.000000,13.750000,1.0,1.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,385.000000,266.666667,416.666667,390.000000
40161,116.666667,234.583333,16.250000,10.000000,1.0,10.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,388.750000,266.666667,416.666667,405.000000
40162,116.666667,231.250000,6.250000,12.083333,1.0,2.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,394.166667,266.666667,416.666667,400.416667
40163,116.666667,245.416667,40.416667,18.750000,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,273.750000,266.666667,416.666667,314.166667


In [0]:
df_annots_new_3.bbox_right_x = df_annots_new_3.bbox_right_x - df_annots_new_3.bbox_left
df_annots_new_3.bbox_center = df_annots_new_3.bbox_center - df_annots_new_3.bbox_left
df_annots_new_3.bbox_right = df_annots_new_3.bbox_right - df_annots_new_3.bbox_left
df_annots_new_3.bbox_left_x = df_annots_new_3.bbox_left_x - df_annots_new_3.bbox_left

In [0]:
df_annots_new_3.bbox_left = df_annots_new_3.bbox_left - df_annots_new_3.bbox_left

In [0]:
df_annots_new_3

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
2,0.0,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,227.777778,150.0,300.0,262.777778
3,0.0,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,205.000000,150.0,300.0,238.888889
6,0.0,281.111111,7.222222,11.111111,1.0,2.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,51.666667,150.0,300.0,58.888889
7,0.0,282.777778,9.444444,16.666667,1.0,10.0,1.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,50.555556,150.0,300.0,60.000000
8,0.0,276.666667,45.555556,23.333333,1.0,4.0,1.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,171.111111,150.0,300.0,216.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40160,0.0,223.333333,5.000000,13.750000,1.0,1.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,268.333333,150.0,300.0,273.333333
40161,0.0,234.583333,16.250000,10.000000,1.0,10.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,272.083333,150.0,300.0,288.333333
40162,0.0,231.250000,6.250000,12.083333,1.0,2.0,0.0,1.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,277.500000,150.0,300.0,283.750000
40163,0.0,245.416667,40.416667,18.750000,1.0,4.0,0.0,0.0,0000316_01201_d_0000525.txt,533.333333,300.0,0.416667,157.083333,150.0,300.0,197.500000


In [0]:
df_annots_new_4 = df_annots_new_3.copy()

In [0]:
df_func_annots_copy = df_func_annots.copy()

In [0]:
df_annots_new_4.head()

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
2,0.0,247.222222,35.000000,19.444444,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,227.777778,150.0,300.0,262.777778
3,0.0,225.555556,33.888889,16.666667,1.0,4.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,205.000000,150.0,300.0,238.888889
6,0.0,281.111111,7.222222,11.111111,1.0,2.0,0.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,51.666667,150.0,300.0,58.888889
7,0.0,282.777778,9.444444,16.666667,1.0,10.0,1.0,1.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,50.555556,150.0,300.0,60.000000
8,0.0,276.666667,45.555556,23.333333,1.0,4.0,1.0,0.0,0000249_00001_d_0000001.txt,533.333333,300.0,0.555556,171.111111,150.0,300.0,216.666667


In [0]:
df_annots_new_5 = pd.DataFrame()
df_annots_new_5 = df_annots_new_4[['bbox_left_x','bbox_top','bbox_width','bbox_height','name_file','object_category']]
df_annots_new_5.head()

,bbox_left_x,bbox_top,bbox_width,bbox_height,name_file,object_category
2,227.777778,247.222222,35.000000,19.444444,0000249_00001_d_0000001.txt,4.0
3,205.000000,225.555556,33.888889,16.666667,0000249_00001_d_0000001.txt,4.0
6,51.666667,281.111111,7.222222,11.111111,0000249_00001_d_0000001.txt,2.0
7,50.555556,282.777778,9.444444,16.666667,0000249_00001_d_0000001.txt,10.0
8,171.111111,276.666667,45.555556,23.333333,0000249_00001_d_0000001.txt,4.0


In [0]:
df_func_annots_copy_1 = pd.DataFrame()
df_func_annots_copy_1 = df_func_annots_copy[['bbox_left','bbox_top','bbox_width','bbox_height','name_file','classes']]
df_func_annots_copy_1.head()

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes
0,142.727142,141.413956,6.645656,18.443022,0000249_00001_d_0000001.txt,1
1,253.247757,116.341927,4.338062,10.542637,0000249_00001_d_0000001.txt,1
2,195.777939,131.615875,8.399105,16.158110,0000249_00001_d_0000001.txt,1
3,284.914856,102.763924,4.213929,10.202283,0000249_00001_d_0000001.txt,1
4,52.854851,73.356804,29.292780,12.229389,0000249_00001_d_0000001.txt,14


In [0]:
for i in range(len(df_annots_new_5)):
  if (df_annots_new_5.object_category.iloc[i] in np.array([5,7,8,10])):
    df_annots_new_5.object_category.iloc[i] = 'other_vehicle'
  if (df_annots_new_5.object_category.iloc[i] in np.array([1,2])):
    df_annots_new_5.object_category.iloc[i] = 'people'
  if(df_annots_new_5.object_category.iloc[i]== 3):
    df_annots_new_5.object_category.iloc[i] = 'bycycle' 
  if(df_annots_new_5.object_category.iloc[i]== 6):
    df_annots_new_5.object_category.iloc[i] = 'trunck'
  if(df_annots_new_5.object_category.iloc[i]== 9):
    df_annots_new_5.object_category.iloc[i] = 'bus'
  if(df_annots_new_5.object_category.iloc[i]== 0):
    df_annots_new_5.object_category.iloc[i] = 'ignored_regions'  
  if(df_annots_new_5.object_category.iloc[i]== 4):
    df_annots_new_5.object_category.iloc[i] = 'сar'
  if(df_annots_new_5.object_category.iloc[i]== 11):
    df_annots_new_5.object_category.iloc[i] = 'other_1'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/pyt

In [0]:
a = np.array(range(3,80))
a = np.delete(a, [2,4])
a

array([ 3,  4,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79])

In [0]:
for i in range(len(df_func_annots_copy_1)):
  if(df_func_annots_copy_1.classes.iloc[i] in a):
    df_func_annots_copy_1.classes.iloc[i] = 'other'
  if(df_func_annots_copy_1.classes.iloc[i]== 0):
    df_func_annots_copy_1.classes.iloc[i] = 'people'
  if(df_func_annots_copy_1.classes.iloc[i]== 1):
    df_func_annots_copy_1.classes.iloc[i] = 'bycycle'
  if(df_func_annots_copy_1.classes.iloc[i]== 7):
    df_func_annots_copy_1.classes.iloc[i] = 'trunck'
  if(df_func_annots_copy_1.classes.iloc[i]== 5):
    df_func_annots_copy_1.classes.iloc[i] = 'bus'
  if(df_func_annots_copy_1.classes.iloc[i]== 2):
    df_func_annots_copy_1.classes.iloc[i] = 'сar'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we c

In [0]:
def max_in_list(lst):
  assert lst
  m = lst[0]
  for i in lst:
    if i > m:
      m = i
  return m


In [0]:
data1 = []
for f in txt_list:
  data1.append(f[f.rindex('/')+1:])

In [0]:
def IoU_1(out, inp):

  box_in = inp
  box_out = out
  for_return = pd.DataFrame()
  for_return_1 = []
  for_return_2 = []

  for i in range(len(box_out)):
    x_out = box_out.iloc[i,0]
    y_out = box_out.iloc[i,1]
    indx_in = []
    peresec = []
    name_out = box_out.iloc[i,4]

    for j in range(len(box_in)):
      name_in = box_in.iloc[j,4]
      if (name_out == name_in):
        width_out = box_out.iloc[i,2]
        height_out = box_out.iloc[i,3]

        width_in = box_in.iloc[j,2]
        height_in = box_in.iloc[j,3]

        x1_in = box_in.iloc[j,0]
        x2_in = box_in.iloc[j,0] + width_in
          
        y1_in = box_in.iloc[j,1]
        y2_in = box_in.iloc[j,1] - height_in
        
        x1_out = box_out.iloc[i,0]
        x2_out = box_out.iloc[i,0] + width_out
        
        y1_out = box_out.iloc[i,1]
        y2_out = box_out.iloc[i,1] - height_out
        
        xA = max(x1_out, x1_in)
        yA = max(y2_out, y2_in)
        xB = min(x2_out, x2_in)
        yB = min(y1_out, y1_in)

        w_p = xB - xA
        h_p = yB - yA

        if(w_p > 0):
          if(h_p > 0):
            interArea = w_p * h_p
          else:
            interArea = 0
        else:
          interArea = 0

        boxAArea = width_out * height_out
        boxBArea = height_in * width_in
        
        iou = interArea / float(boxAArea + boxBArea - interArea)
        indx_in.append(j)
        peresec.append(iou)

    if(peresec): 
      m_iou = max_in_list(peresec)
      for q in range(len(peresec)):
        if(peresec[q] == m_iou):
          index = indx_in[q]
          m_cl = box_in.iloc[index,5]
    else:
      peresec.append(0)
      m_iou = 0
      m_cl = 'nothing'
    
    
    for_return_1.append(m_iou)
    for_return_2.append(m_cl)

    print(i)
    print(m_iou)
    print('----------------------')
    
  for_return['IoU'] = pd.Series(for_return_1)
  for_return['class_in'] = pd.Series(for_return_2)
  return for_return

In [0]:
df_func_annots_copy_2 = pd.DataFrame()
df_annots_new_5_copy = pd.DataFrame()

In [0]:
df_func_annots_copy_2 = df_func_annots_copy_1.head(200)

In [0]:
df_annots_new_5_copy = df_annots_new_5.head(1000)

In [0]:
import math
import matplotlib.patches as patches
import matplotlib.pyplot as plt

In [0]:
IOU_CL = IoU_1(df_func_annots_copy_1,df_annots_new_5)

715
0.6995090885654651
----------------------
716
0.12455783470677295
----------------------
717
0.17978611469291886
----------------------
718
0.006808712339735733
----------------------
719
0.09681285997125366
----------------------
720
0.0
----------------------
721
0.010029283268775457
----------------------
722
0.0
----------------------
723
0.08986124257780974
----------------------
724
0.01107048992588955
----------------------
725
0.007398781552241285
----------------------
726
0.08197091786571269
----------------------
727
0.006187220973296962
----------------------
728
0.05839137867467805
----------------------
729
0.07722343661543785
----------------------
730
0.0013332913699862427
----------------------
731
0.0
----------------------
732
0.09002974113441942
----------------------
733
0.059206247626284556
----------------------
734
0.03700009327728279
----------------------
735
0.05204037230101331
----------------------
736
0.0012620518353497797
----------------------
737
0.

KeyboardInterrupt: ignored

In [0]:
IOU_CL.to_csv('/content/gdrive/My Drive/IOU_CL.csv', index = False, header=True)

In [12]:
IOU_CL

,IoU,class_in
0,0.000000,ignored_regions
1,0.000000,ignored_regions
2,0.000000,ignored_regions
3,0.000000,ignored_regions
4,0.000000,ignored_regions
...,...,...
10767,0.000000,ignored_regions
10768,0.000000,ignored_regions
10769,0.085312,ignored_regions
10770,0.000000,ignored_regions


In [0]:
df_func_annots_copy_1 = pd.DataFrame()
df_func_annots_copy_1 = df_func_annots_copy
df_func_annots_copy_1.head()

In [0]:
for i in range(len(df_func_annots_copy_1)):
  if(df_func_annots_copy_1.classes.iloc[i] in a):
    df_func_annots_copy_1.classes.iloc[i] = 'other'
  if(df_func_annots_copy_1.classes.iloc[i]== 0):
    df_func_annots_copy_1.classes.iloc[i] = 'people'
  if(df_func_annots_copy_1.classes.iloc[i]== 1):
    df_func_annots_copy_1.classes.iloc[i] = 'bycycle'
  if(df_func_annots_copy_1.classes.iloc[i]== 7):
    df_func_annots_copy_1.classes.iloc[i] = 'trunck'
  if(df_func_annots_copy_1.classes.iloc[i]== 5):
    df_func_annots_copy_1.classes.iloc[i] = 'bus'
  if(df_func_annots_copy_1.classes.iloc[i]== 2):
    df_func_annots_copy_1.classes.iloc[i] = 'сar'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
Final_df = df_func_annots_copy_1

In [0]:
Final_df['IoU'] = IOU_CL['IoU']

In [0]:
Final_df['class_in'] = IOU_CL['class_in']

In [0]:
Final_df.to_csv('/content/gdrive/My Drive/Final_df.csv', index = False, header=True)

In [51]:
Final_df

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes,IoU,class_in
0,5.985657,106.613680,8.662201,11.500418,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
1,188.913220,86.912030,6.454289,16.877890,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
2,213.319430,61.774788,3.999245,16.105307,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
3,11.917746,113.200120,15.395401,30.303400,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
4,4.939372,128.905290,26.134935,16.557777,0000249_03900_d_0000011.txt,сar,0.000000,ignored_regions
...,...,...,...,...,...,...,...,...
10767,199.896030,60.889133,13.895166,14.755516,0000359_00785_d_0000703.txt,other,0.000000,ignored_regions
10768,78.957100,266.935360,46.184370,30.716587,0000359_00785_d_0000703.txt,other,0.000000,ignored_regions
10769,110.667950,73.021454,78.561966,74.463170,0000359_00785_d_0000703.txt,other,0.085312,ignored_regions
10770,118.306380,153.480270,22.223675,23.065996,0000359_00785_d_0000703.txt,other,0.000000,ignored_regions


In [0]:
Final_df_new = Final_df.copy()

In [53]:
Final_df_new

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes,IoU,class_in
0,5.985657,106.613680,8.662201,11.500418,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
1,188.913220,86.912030,6.454289,16.877890,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
2,213.319430,61.774788,3.999245,16.105307,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
3,11.917746,113.200120,15.395401,30.303400,0000249_03900_d_0000011.txt,bycycle,0.000000,ignored_regions
4,4.939372,128.905290,26.134935,16.557777,0000249_03900_d_0000011.txt,сar,0.000000,ignored_regions
...,...,...,...,...,...,...,...,...
10767,199.896030,60.889133,13.895166,14.755516,0000359_00785_d_0000703.txt,other,0.000000,ignored_regions
10768,78.957100,266.935360,46.184370,30.716587,0000359_00785_d_0000703.txt,other,0.000000,ignored_regions
10769,110.667950,73.021454,78.561966,74.463170,0000359_00785_d_0000703.txt,other,0.085312,ignored_regions
10770,118.306380,153.480270,22.223675,23.065996,0000359_00785_d_0000703.txt,other,0.000000,ignored_regions


In [0]:
Final_df_new['err_class'] = 0 

In [55]:
for i in range(len(Final_df_new)):
  if(Final_df_new.classes.iloc[i] != Final_df_new.class_in.iloc[i]):
    Final_df_new.err_class.iloc[i] = 0
  else:
    Final_df_new.err_class.iloc[i] = 1


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
Final_df_new_1 = Final_df_new

In [62]:
Final_df_new_1.head()

,bbox_left,bbox_top,bbox_width,bbox_height,name_file,classes,IoU,class_in,err_class
0,5.985657,106.613680,8.662201,11.500418,0000249_03900_d_0000011.txt,bycycle,0.0,ignored_regions,0
1,188.913220,86.912030,6.454289,16.877890,0000249_03900_d_0000011.txt,bycycle,0.0,ignored_regions,0
2,213.319430,61.774788,3.999245,16.105307,0000249_03900_d_0000011.txt,bycycle,0.0,ignored_regions,0
3,11.917746,113.200120,15.395401,30.303400,0000249_03900_d_0000011.txt,bycycle,0.0,ignored_regions,0
4,4.939372,128.905290,26.134935,16.557777,0000249_03900_d_0000011.txt,сar,0.0,ignored_regions,0


In [61]:
df_annots_new_3.head()

,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,truncation,occlusion,name_file,width,height,coeff,bbox_left_x,bbox_center,bbox_right,bbox_right_x
0,0.0,232.222222,37.777778,15.555556,1.0,4.0,0.0,1.0,0000249_03900_d_0000011.txt,533.333333,300.0,0.555556,11.666667,150.0,300.0,49.444444
1,0.0,204.444444,23.888889,17.777778,1.0,5.0,0.0,0.0,0000249_03900_d_0000011.txt,533.333333,300.0,0.555556,63.888889,150.0,300.0,87.777778
2,0.0,191.111111,23.888889,14.444444,1.0,4.0,0.0,1.0,0000249_03900_d_0000011.txt,533.333333,300.0,0.555556,64.444444,150.0,300.0,88.333333
3,0.0,183.333333,26.666667,11.111111,1.0,4.0,0.0,0.0,0000249_03900_d_0000011.txt,533.333333,300.0,0.555556,93.888889,150.0,300.0,120.555556
4,0.0,167.222222,2.777778,7.222222,1.0,10.0,0.0,0.0,0000249_03900_d_0000011.txt,533.333333,300.0,0.555556,105.555556,150.0,300.0,108.333333


In [0]:
task_4_df = Final_df_new_1[Final_df_new_1['IoU']>= 0.35]

In [0]:
acc_35 = sum(task_4_df.err_class==1)/len(task_4_df)

In [0]:
fail_35 = len(task_4_df) - sum(task_4_df.err_class==1)

In [0]:
T4 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_4_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_4_df.name_file.iloc[j]):
      T4 = T4.append(df_annots_new_3.iloc[i])

In [0]:
loss_35 = len(T4) - sum(task_4_df.err_class==1)

In [0]:
task_4_df_1 = Final_df_new_1[Final_df_new_1['IoU']>= 0.30]
acc_30 = sum(task_4_df_1.err_class==1)/len(task_4_df_1)
fail_30 = len(task_4_df_1) - sum(task_4_df_1.err_class==1)

In [0]:
T4_1 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_4_df_1)):
    if(df_annots_new_3.name_file.iloc[i]==task_4_df_1.name_file.iloc[j]):
      T4_1 = T4_1.append(df_annots_new_3.iloc[i])

In [0]:
loss_30 = len(T4_1) - sum(task_4_df_1.err_class==1)

In [0]:
task_4_df_2 = Final_df_new_1[Final_df_new_1['IoU']>= 0.40]
acc_40 = sum(task_4_df_2.err_class==1)/len(task_4_df_2)
fail_40 = len(task_4_df_2) - sum(task_4_df_2.err_class==1)

In [0]:
T4_2 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_4_df_2)):
    if(df_annots_new_3.name_file.iloc[i]==task_4_df_2.name_file.iloc[j]):
      T4_2 = T4_2.append(df_annots_new_3.iloc[i])

In [0]:
loss_40 = len(T4_2) - sum(task_4_df_2.err_class==1)

In [0]:
TASK4 = pd.DataFrame(columns=['IoU','acc','loss','fail'])
TASK4 = TASK4.append({'IoU': 0.30, 'acc':acc_30*100, 'loss': loss_30, 'fail': fail_30}, ignore_index=True)
TASK4 = TASK4.append({'IoU': 0.35, 'acc':acc_35*100, 'loss': loss_35, 'fail': fail_35}, ignore_index=True)
TASK4 = TASK4.append({'IoU': 0.40, 'acc':acc_40*100, 'loss': loss_40, 'fail': fail_40}, ignore_index=True)

In [112]:
TASK4

,IoU,acc,loss,fail
0,0.30,0.539084,22236.0,369.0
1,0.35,0.403226,14364.0,247.0
2,0.40,0.645161,9434.0,154.0


In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.35)&(Final_df_new_1['class_in']=='people')]

In [0]:
acc_35_people = sum(task_5_df.err_class==1)/len(task_5_df)

In [0]:
fail_35_people = len(task_5_df) - sum(task_5_df.err_class==1)

In [0]:
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])

In [0]:
loss_35_people = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()

In [0]:
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.35)&(Final_df_new_1['class_in']=='bycycle')]

In [0]:
acc_35_bycycle = sum(task_5_df.err_class==1)/len(task_5_df)
fail_35_bycycle = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_35_bycycle = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.35)&(Final_df_new_1['class_in']=='car')]

In [0]:
acc_30_car = 0
fail_30_car = 0
loss_30_car = 0
acc_35_car = 0
fail_35_car = 0
loss_35_car = 0
acc_40_car = 0
fail_40_car = 0
loss_40_car = 0

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.35)&(Final_df_new_1['class_in']=='trunc')]

In [0]:
acc_30_trunc = 0
fail_30_trunc = 0
loss_30_trunc = 0
acc_35_trunc = 0
fail_35_trunc = 0
loss_35_trunc = 0
acc_40_trunc = 0
fail_40_trunc = 0
loss_40_trunc = 0

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.35)&(Final_df_new_1['class_in']=='bus')]

In [0]:
acc_35_bus = sum(task_5_df.err_class==1)/len(task_5_df)
fail_35_bus = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_35_bus = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.30)&(Final_df_new_1['class_in']=='people')]

In [0]:
acc_30_people = sum(task_5_df.err_class==1)/len(task_5_df)
fail_30_people = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_30_people = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.40)&(Final_df_new_1['class_in']=='people')]

In [0]:
acc_40_people = sum(task_5_df.err_class==1)/len(task_5_df)
fail_40_people = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_40_people = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.30)&(Final_df_new_1['class_in']=='bycycle')]

In [0]:
acc_30_bycycle = sum(task_5_df.err_class==1)/len(task_5_df)
fail_30_bycycle = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_30_bycycle = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.40)&(Final_df_new_1['class_in']=='bycycle')]

In [0]:
acc_40_bycycle = sum(task_5_df.err_class==1)/len(task_5_df)
fail_40_bycycle = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_40_bycycle = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.30)&(Final_df_new_1['class_in']=='bus')]

In [0]:
acc_30_bus = sum(task_5_df.err_class==1)/len(task_5_df)
fail_30_bus = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_30_bus = len(T5) - sum(task_5_df.err_class==1)

In [0]:
task_5_df = pd.DataFrame()
task_5_df = Final_df_new_1[(Final_df_new_1['IoU']>= 0.40)&(Final_df_new_1['class_in']=='bus')]

In [0]:
acc_40_bus = sum(task_5_df.err_class==1)/len(task_5_df)
fail_40_bus = len(task_5_df) - sum(task_5_df.err_class==1)
T5 = pd.DataFrame()
for i in range(len(df_annots_new_3)):
  for j in range(len(task_5_df)):
    if(df_annots_new_3.name_file.iloc[i]==task_5_df.name_file.iloc[j]):
      T5 = T5.append(df_annots_new_3.iloc[i])
loss_40_bus = len(T5) - sum(task_5_df.err_class==1)

In [0]:
TASK5 = pd.DataFrame(columns=['class','IoU','acc','loss','fail'])
TASK5 = TASK5.append({'class':'people','IoU': 0.30, 'acc':acc_30_people*100, 'loss': loss_30_people, 'fail': fail_30_people}, ignore_index=True)
TASK5 = TASK5.append({'class':'people','IoU': 0.35, 'acc':acc_35_people*100, 'loss': loss_35_people, 'fail': fail_35_people}, ignore_index=True)
TASK5 = TASK5.append({'class':'people','IoU': 0.40, 'acc':acc_40_people*100, 'loss': loss_40_people, 'fail': fail_40_people}, ignore_index=True)
TASK5 = TASK5.append({'class':'bicycle','IoU': 0.30, 'acc':acc_30_bycycle*100, 'loss': loss_30_bycycle, 'fail': fail_30_bycycle}, ignore_index=True)
TASK5 = TASK5.append({'class':'bicycle','IoU': 0.35, 'acc':acc_35_bycycle*100, 'loss': loss_35_bycycle, 'fail': fail_35_bycycle}, ignore_index=True)
TASK5 = TASK5.append({'class':'bicycle','IoU': 0.40, 'acc':acc_40_bycycle*100, 'loss': loss_40_bycycle, 'fail': fail_40_bycycle}, ignore_index=True)
TASK5 = TASK5.append({'class':'car','IoU': 0.30, 'acc':acc_30_car*100, 'loss': loss_30_car, 'fail': fail_30_car}, ignore_index=True)
TASK5 = TASK5.append({'class':'car','IoU': 0.35, 'acc':acc_35_car*100, 'loss': loss_35_car, 'fail': fail_35_car}, ignore_index=True)
TASK5 = TASK5.append({'class':'car','IoU': 0.40, 'acc':acc_40_car*100, 'loss': loss_40_car, 'fail': fail_40_car}, ignore_index=True)
TASK5 = TASK5.append({'class':'bus','IoU': 0.30, 'acc':acc_30_bus*100, 'loss': loss_30_bus, 'fail': fail_30_bus}, ignore_index=True)
TASK5 = TASK5.append({'class':'bus','IoU': 0.35, 'acc':acc_35_bus*100, 'loss': loss_35_bus, 'fail': fail_35_bus}, ignore_index=True)
TASK5 = TASK5.append({'class':'bus','IoU': 0.40, 'acc':acc_40_bus*100, 'loss': loss_40_bus, 'fail': fail_40_bus}, ignore_index=True)
TASK5 = TASK5.append({'class':'trunc','IoU': 0.30, 'acc':acc_30_trunc*100, 'loss': loss_30_trunc, 'fail': fail_30_trunc}, ignore_index=True)
TASK5 = TASK5.append({'class':'trunc','IoU': 0.35, 'acc':acc_35_trunc*100, 'loss': loss_35_trunc, 'fail': fail_35_trunc}, ignore_index=True)
TASK5 = TASK5.append({'class':'trunc','IoU': 0.40, 'acc':acc_40_trunc*100, 'loss': loss_40_trunc, 'fail': fail_40_trunc}, ignore_index=True)

In [186]:
TASK5

,class,IoU,acc,loss,fail
0,people,0.30,0.0,1977,36
1,people,0.35,0.0,1281,24
2,people,0.40,0.0,901,15
3,bicycle,0.30,0.0,324,6
4,bicycle,0.35,0.0,126,2
5,bicycle,0.40,0.0,63,1
6,car,0.30,0.0,0,0
7,car,0.35,0.0,0,0
8,car,0.40,0.0,0,0
9,bus,0.30,0.0,433,7


In [0]:
for image_idx in np.arange(130,135,1):
    fig, ax = plt.subplots(1)
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        x = bboxes[idx][0]
        y = bboxes[idx][1]
        w = bboxes[idx][2]
        h = bboxes[idx][3]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    box = pd.DataFrame()
    name = data1[image_idx]
    print(name)
    box = df_annots_new_5.loc[df_annots_new_5.name_file == name]
    for idx in range(len(box)):
        x1 = box.bbox_left.iloc[idx]
        x2 = box.bbox_top.iloc[idx]
        x3 = box.bbox_width.iloc[idx]
        x4 = box.bbox_height.iloc[idx]
        rect1 = patches.Rectangle((x1, x2), x3, x4, linewidth=1, edgecolor='blue', facecolor='none')
        ax.add_patch(rect1)
plt.show()